In [1]:
# install.packages('DirichletReg')

In [26]:
library(Seurat)
library(RColorBrewer) #for brewer.pal
library(Matrix) #for Matrix
library(DirichletReg)
library(data.table)
library(tidyverse)
library(cowplot)

## this function is extracted from analysis.r 
dirichlet_regression = function(counts, covariates, formula){  
  # Dirichlet multinomial regression to detect changes in cell frequencies
  # formula is not quoted, example: counts ~ condition
  # counts is a [samples x cell types] matrix
  # covariates holds additional data to use in the regression
  #
  # Example:
  # counts = do.call(cbind, tapply(seur@data.info$orig.ident, seur@ident, table))
  # covariates = data.frame(condition=gsub('[12].*', '', rownames(counts)))
  # res = dirichlet_regression(counts, covariates, counts ~ condition)
  
  #ep.pvals = dirichlet_regression(counts=ep.freq, covariates=ep.cov, formula=counts ~ condition)$pvals

  # Calculate regression
  counts = as.data.frame(counts)
  counts$counts = DR_data(counts)
  data = cbind(counts, covariates)
  fit = DirichReg(counts ~ condition, data) 
  
  # Get p-values
  u = summary(fit)
  #compared with healthy condition, 15 vars. noninflame and inflame, 30pvalues
  pvals = u$coef.mat[grep('Intercept', rownames(u$coef.mat), invert=T), 4] 
  v = names(pvals)
  pvals = matrix(pvals, ncol=length(u$varnames))
  rownames(pvals) = gsub('condition', '', v[1:nrow(pvals)])
  colnames(pvals) = u$varnames
  fit$pvals = pvals
  
  fit
}

In [15]:
freq=read.csv('celltype.csv',row.names=1)
freq1=as.matrix(as.data.frame(lapply(freq, as.double),row.names=row.names(freq)))
cov=read.csv('conv.csv',row.names=1)
cov1 = data.frame(condition=factor(cov[rownames(freq),1], levels=c('healthy control','mild(moderate)','severe','convalescence','influenza')), 
                  row.names=rownames(freq))

In [16]:
pvals = dirichlet_regression(counts=freq1, covariates=cov1, formula=counts ~ condition)$pvals
colnames(pvals) = colnames(freq1)
# write.csv(pvals,'healthy(control).csv')

Warning message in DR_data(counts):
“not all rows sum up to 1 => normalization forced
  some entries are 0 or 1 => transformation forced”


In [21]:
cov=read.csv('conv.csv',row.names=1)
cov1 = data.frame(condition=factor(cov[rownames(freq),1], levels=c('mild(moderate)','healthy control','severe','convalescence','influenza')), 
                  row.names=rownames(freq))
pvals = dirichlet_regression(counts=freq1, covariates=cov1, formula=counts ~ condition)$pvals
colnames(pvals) = colnames(freq1)
# write.csv(pvals,'mild(control).csv')

Warning message in DR_data(counts):
“not all rows sum up to 1 => normalization forced
  some entries are 0 or 1 => transformation forced”


In [2]:
sessionInfo()

R version 4.0.2 (2020-06-22)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.6 LTS

Matrix products: default
BLAS:   /usr/lib/libblas/libblas.so.3.6.0
LAPACK: /usr/lib/lapack/liblapack.so.3.6.0

locale:
 [1] LC_CTYPE=C.UTF-8           LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] cowplot_1.1.1      forcats_0.5.1      stringr_1.4.0      dplyr_1.0.4       
 [5] purrr_0.3.4        readr_1.4.0        tidyr_1.1.2        tibble_3.0.6      
 [9] ggplot2_3.3.3      tidyverse_1.3.0    data.table_1.13.6  DirichletReg_0.7-0
[13] Formula_1.2-4      Matrix_1.2-18      RColorBrewer_